In [ ]:
import torch

'''
================================================
加载模型
================================================
'''
gpu = torch.device(f'cuda:0' if torch.cuda.is_available() else 'cpu')
net = torch.load('../input/trainedmodel/res.pt', map_location=gpu)

# print(next(net.parameters()).device)

In [ ]:
import glob
import random
import numpy as np
import cv2
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
import imgaug as ia
from imgaug import augmenters as iaa
from matplotlib import pyplot as plt

import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms

path = '../input/rsna-miccai-brain-tumor-radiogenomic-classification'

'''
================================================
加载数据：读取dicom数据->数据增强->每个个人的四种数据合成一个样本->Dataset->DataLoader
================================================
'''
'''1-1, 数据增强'''
sometimes = lambda aug: iaa.Sometimes(0.2, aug)
seq = iaa.Sequential(
    [
        iaa.Fliplr(0.5),    #水平翻转
        iaa.Flipud(0.2),    #竖直翻转
        sometimes(iaa.CropAndPad(
            percent=(-0.05, 0.05),  # crop images by -5% to 10% of their height/width
            pad_mode=ia.ALL,
            pad_cval=(0, 255)
        )),
        sometimes(iaa.Affine(
            scale={"x": (0.8, 1.2), "y": (0.8, 1.2)},   # scale images to 80-120% of their size, individually per axis
            translate_percent={"x": (-0.2, 0.2), "y": (-0.2, 0.2)}, # translate by -20 to +20 percent (per axis)
            rotate=(-45, 45),   # rotate by -45 to +45 degrees
            shear=(-16, 16),    # shear by -16 to +16 degrees
            order=[0, 1],   # use nearest neighbour or bilinear interpolation (fast)
            cval=(0, 255),  # if mode is constant, use a cval between 0 and 255
            mode=ia.ALL     #use any of scikit-image's warping modes (see 2nd image from the top for examples)
        )),
        # execute 0 to 5 of the following (less important) augmenters per image
        # don't execute all of them, as that would often be way too strong
        iaa.SomeOf((0, 5),
                   [
                       sometimes(iaa.Superpixels(p_replace=(0, 1.0), n_segments=(20, 200))),
                       # convert images into their superpixel representation
                       iaa.OneOf([
                           iaa.GaussianBlur((0, 3.0)),  # blur images with a sigma between 0 and 3.0
                           iaa.AverageBlur(k=(2, 7)),  # blur image using local means with kernel sizes between 2 and 7
                           iaa.MedianBlur(k=(3, 11)),
                           # blur image using local medians with kernel sizes between 2 and 7
                       ]),
                       iaa.Sharpen(alpha=(0, 1.0), lightness=(0.75, 1.5)),  # sharpen images
                       iaa.Emboss(alpha=(0, 1.0), strength=(0, 2.0)),  # emboss images
                       # search either for all edges or for directed edges,
                       # blend the result with the original image using a blobby mask
                       iaa.SimplexNoiseAlpha(iaa.OneOf([
                           iaa.EdgeDetect(alpha=(0.5, 1.0)),
                           iaa.DirectedEdgeDetect(alpha=(0.5, 1.0), direction=(0.0, 1.0)),
                       ])),
                       iaa.AdditiveGaussianNoise(loc=0, scale=(0.0, 0.05 * 255), per_channel=0.5),
                       # add gaussian noise to images
                       iaa.OneOf([
                           iaa.Dropout((0.01, 0.1), per_channel=0.5),  # randomly remove up to 10% of the pixels
                           iaa.CoarseDropout((0.03, 0.15), size_percent=(0.02, 0.05), per_channel=0.2),
                       ]),
                       iaa.Invert(0.05, per_channel=True),  # invert color channels
                       iaa.Add((-10, 10), per_channel=0.5),
                       # change brightness of images (by -10 to 10 of original value)

                       # either change the brightness of the whole image (sometimes
                       # per channel) or change the brightness of subareas
                       iaa.OneOf([
                           iaa.Multiply((0.5, 1.5), per_channel=0.5),
                           iaa.FrequencyNoiseAlpha(
                               exponent=(-4, 0),
                               first=iaa.Multiply((0.5, 1.5), per_channel=True),
                               second=iaa.LinearContrast((0.5, 2.0))
                           )
                       ]),
                       iaa.LinearContrast((0.5, 2.0), per_channel=0.5),  # improve or worsen the contrast
                       sometimes(iaa.ElasticTransformation(alpha=(0.5, 3.5), sigma=0.25)),
                       # move pixels locally around (with random strengths)
                       sometimes(iaa.PiecewiseAffine(scale=(0.01, 0.05))),  # sometimes move parts of the image around
                       sometimes(iaa.PerspectiveTransform(scale=(0.01, 0.1)))
                   ],
                   random_order=True
                   )
    ],
    random_order=True
)

'''1-2, 将dcm数据转换为numpy.array，利用seq并进行数据增强'''
def dicom2array(paths, voi_lut=True, fix_monochrome=True, remove_black_boundary=True, aug=False):
    for path in paths:
        dicom = pydicom.read_file(path)
        # VOI LUT (if available by DICOM device) is used to
        # transform raw DICOM data to "human-friendly" view
        if voi_lut:
            data = apply_voi_lut(dicom.pixel_array, dicom)
        else:
            data = dicom.pixel_array
        if data.max() > 0.0:  # avoiding black images (if possible)
            break
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    if remove_black_boundary:  # we get slightly more details
        (x, y) = np.where(data > 0)
        if len(x) > 0 and len(y) > 0:
            x_mn = np.min(x)
            x_mx = np.max(x)
            y_mn = np.min(y)
            y_mx = np.max(y)
            if (x_mx - x_mn) > 10 and (y_mx - y_mn) > 10:
                data = data[:, np.min(y):np.max(y)]
    data = cv2.resize(data, (512, 512))
    if aug:
        data = seq(images=data)
    return data

'''1-3, 借助于dicom2array加载数据，并且将每一个病患的四类数据合成一个样本'''
def load_rand_dicom_images(scan_id, split="train", aug=False):
    """
    send 4 random slices of each modality
    """
    if split != "train" and split != "test":
        split = "train"
    flair = sorted(glob.glob(f"{path}/{split}/{scan_id}/FLAIR/*.dcm"))
    flair_img = dicom2array(random.sample(flair, max(len(flair) // 2, 1)), aug=aug)
    t1w = sorted(glob.glob(f"{path}/{split}/{scan_id}/T1w/*.dcm"))
    t1w_img = dicom2array(random.sample(t1w, max(len(t1w) // 2, 1)), aug=aug)
    t1wce = sorted(glob.glob(f"{path}/{split}/{scan_id}/T1wCE/*.dcm"))
    t1wce_img = dicom2array(random.sample(t1wce, max(len(t1wce) // 2, 1)), aug=aug)
    t2w = sorted(glob.glob(f"{path}/{split}/{scan_id}/T2w/*.dcm"))
    t2w_img = dicom2array(random.sample(t2w, max(len(t2w) // 2, 1)), aug=aug)

    return np.array((flair_img, t1w_img, t1wce_img, t2w_img)).T

# '''测试'''
# data = load_rand_dicom_images("00001", split='test')
# print(data.shape)
# for i in range(4):
#     plt.imshow(data[:, :, i])
#     plt.pause(0.01)
# plt.show()

'''1-4,Dataset'''
class BrainTumor(Dataset):
    def __init__(self):
        super().__init__()
        self.ids = [a.split("/")[-1] for a in sorted(glob.glob(path + f"/test/" + "/*"))]
    def __len__(self):
        return len(self.ids)
    def __getitem__(self, idx):
        imgs = load_rand_dicom_images(self.ids[idx], 'test', aug=False)
        transform = transforms.Compose(
            [transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5, 0.5), (0.5, 0.5, 0.5, 0.5))])
        imgs = transform(imgs)
        return torch.tensor(imgs, dtype=torch.float32)
    
'''1-5, dataloader'''
test_bs = 16
test_dataset = BrainTumor()
test_loader = DataLoader(test_dataset, batch_size=test_bs, num_workers=4)

# '''测试'''
# for img in test_loader:
#     print(img.shape)

In [ ]:
import pandas as pd

'''
================================================
获取结果
================================================
'''
'''加载样例'''
df = pd.read_csv('../input/rsna-miccai-brain-tumor-radiogenomic-classification/sample_submission.csv',dtype="string")
# print(df)
# assert None

'''使用训练好的网络填充结果'''
n = 0
for data in test_loader:
    res = net(data.to(gpu))
#     print(res)
#     print((res[:, 0] / res.sum(dim=1)).detach().numpy())
    df['MGMT_value'][n*test_bs: (n+1)*test_bs] = [f'{i}' for i in (res[:, 0] / res.sum(dim=1)).detach().cpu().numpy()]
    n+=1
#     print(df)

'''保存'''
df.to_csv('submission.csv', index = False)

In [ ]:
print(pd.read_csv('submission.csv'))